# **Perceptrons** 

Lab designed by **Professor Christopher Yau** University of Manchester.

## Overview

This practical is designed to illustrate the construction of a perceptron algorithm for a binary classification task. In this exercise you will:

1. Use Python and libraries including numpy (numerical computations) and matplotlib (visualisaton).
2. Learn to simulate an artificial dataset to use as test data for the classifer.
3. Create functions in Python to perform the individuals steps of the Perceptron algorithm.
4. Write a complete perceptron algorithm using all the functions in [3].
5. Visualise and plot the outputs.

## Pre-requisities

This lab exercise assumes some basic programming familiarity with Python. The exercise is designed to avoid excessive use of custom libraries and tries to use as much of standard Python as possible. The exemplar code therefore is designed for readability and interpretability (rather than efficiency) to help you to learn about how the Perceptron works. 

## Getting Started
First, we will import some libraries that will be useful later.

In [ ]:
import matplotlib.pyplot as plt # for visualisation
import random # for random number generation
import numpy as np # for numerical libraries

random.seed(242785) # seed the random number generators
np.random.seed(64254)

## Simulating data

In this part of the code, we will simulate some data using the generative model that underlies the perceptron algorithm.

First, we will specify some parameters $\bf{w}$ for our model:

In [ ]:
w0 = 0 # parameter values used to generate data
w1 = -1.5
w2 = 2.5

Normally, in an experiment or classification tasks, the input or predictors would be given. But here, we need to generate some input/predictor values $(x_1, x_2)$. We will do this by sampling random vectors from a multivariate normal distribution (https://en.wikipedia.org/wiki/Multivariate_normal_distribution).

In [ ]:
n = 500 # number of training samples to generate

In [ ]:
mean = [0, 0] # mean
cov = [[3, 0.5], [0.5, 3]] # covariance matrix
x1, x2 = np.random.multivariate_normal(mean, cov, n).T # sample from a multivariate normal distribution

Now recall we next generate latent variables $z$ using our inputs and parameters and convert into outputs $y$ by taking the sign:
$$
  z = w_1 x_1 + w_2 x_2 + w_0
$$
$$
  y = sign(z)
$$

In [ ]:
z = w1*x1 + w2*x2 + w0 # generate the latent variable z
y = np.sign(z) # generate the output y based on the sign of z

Lets plot the data to see how the two classes separate (or not) in $(x_1, x_2)$:

In [ ]:
cdict = {1: 'red', 2: 'blue', 3: 'green'} # colour scheme
fig, ax = plt.subplots()
j = 1
for g in np.unique(y):
    ix = np.where(y == g)
    ax.scatter(x1[ix], x2[ix], c = cdict[j], label = "y = " + str(g), s = 16, alpha=0.3)
    j = j + 1
ax.legend()
plt.xlabel("$x_1$")
plt.ylabel("$x_2$")
plt.show()

The classes do separate so now lets apply our perceptron algorithm and hope it does too.

## Perceptron algorithm

We will now construct the perceptron algorithm to classify this data. First we will define some useful functions. 

The first is a prediction function that takes inputs $(x_1, x_2)$ and parameters $(w_0, w_1, w_2)$ and outputs a prediction $\hat{y}$.

In [ ]:
# define prediction function
def predict_fn(x1, x2, w0, w1, w2):
  y_hat = np.sign(w1*x1 + w2*x2 + w0)
  return y_hat

Next, we define the hinge loss function which involves predicting for all out input samples using some parameters and then counting how many are different from the observed output:
$$
  L_{w}(y,\hat{y}) = -\sum_{i=1}^n y_i \hat{y}_i \mathbb{I}(y_i \neq \hat{y}_i)
$$
where
$
  \hat{y}_i = (w_0 + w_1 x_{1,i} + w_2 x_{2,i})
$ is the output prediction using the current set of parameters $w$ and $\mathbb{I}(\cdot)$ is an indicator function which is 1 if the condition inside the bracket is true and 0 otherwise.

In [ ]:
# definition of loss function
def loss_fn(x1, x2, y, w0, w1, w2):
  y_hat = predict_fn(x1, x2, w0, w1, w2)
  loss = -np.sum( y*y_hat*(y != y_hat) )
  return loss 

Now we define the stochastic gradient descent (SGD) update function, if $(y_i = 1, \hat{y} = -1)$ then we update using:

\begin{align}
  w_0^{(t+1)} &= w_0^{(t)} + 1, ~ w_1^{(t+1)} = w_1^{(t)} + x_{1,i}, ~ w_2^{(t+1)} = w_2^{(t)} + x_{2,i},
\end{align}

else we use

\begin{align}
  w_0^{(t+1)} = w_0^{(t)} - 1, ~ w_1^{(t+1)} = w_1^{(t)} - x_{1,i}, ~ w_2^{(t+1)} = w_2^{(t)} - x_{2,i},
\end{align}

In [ ]:
# define update function
def update_fn(x1, x2, y, y_hat, w0, w1, w2):
  if ( ( y_hat == -1 ) & ( y == 1 ) ) :
    w1 = w1 + x1
    w2 = w2 + x2
    w0 = w0 + 1
  if ( ( y_hat == 1 ) & ( y == -1 ) ) :
    w1 = w1 - x1
    w2 = w2 - x2
    w0 = w0 - 1  
  return w0, w1, w2

We will now specify how many training epochs to use (an epoch is essentially one update step) and the initial parameter estimates:

In [ ]:
n_epochs = 5000 # number of training epochs
w1_hat = -0.5 # initialise parameter estimate
w2_hat = 1.5
w0_hat = 0

Now we begin the perceptron algorithm but first we set up some arrays to hold the parameter and loss values as they change with each update:

In [ ]:
loss_vec = np.zeros(n_epochs)
w0_vec = np.zeros(n_epochs)
w1_vec = np.zeros(n_epochs)
w2_vec = np.zeros(n_epochs)

Now lets iterate the perceptron update steps up to the maximum number of epochs:

In [ ]:
for epoch in range(n_epochs): # for each epoch
  
  # sample one of the training data points
  i = random.randint(0, n-1)
  y_i = y[i]
  x1_i = x1[i]
  x2_i = x2[i]
  
  # predict the output using the current parameters and input
  y_hat_i = predict_fn(x1_i, x2_i, w0_hat, w1_hat, w2_hat) 
  
  # update the parameters
  w0_hat, w1_hat, w2_hat = update_fn(x1_i, x2_i, y_i, y_hat_i, w0_hat, w1_hat, w2_hat)
  
  # compute the new loss 
  loss = loss_fn(x1, x2, y, w0_hat, w1_hat, w2_hat)

  # store
  loss_vec[epoch] = loss
  w0_vec[epoch] = w0_hat
  w1_vec[epoch] = w1_hat
  w2_vec[epoch] = w2_hat

After the perceptron has finished running, we can plot the loss function to check for convergence:

In [ ]:
plt.scatter(np.arange(0, n_epochs), loss_vec, s=16)
plt.xlabel("Epoch")
plt.ylabel("Loss")

Lets see the classification using the true and estimated parameters:

In [ ]:
# compute the predictions for all the training data
y_hat = predict_fn(x1, x2, w0_hat, w1_hat, w2_hat)

In [ ]:
# set up the plots
fig, (ax1, ax2) = plt.subplots(1, 2)
j = 1
for g in np.unique(y):
    ix = np.where(y == g)
    ax1.scatter(x1[ix], x2[ix], c = cdict[j], label = "y = " + str(g), s = 16, alpha=0.3)
    j = j + 1

j = 1
for g in np.unique(y_hat):    
    ix = np.where(y_hat == g)
    ax2.scatter(x1[ix], x2[ix], c = cdict[j], label = "$\hat{y} = $" + str(g), s = 16, alpha=0.3)
    j = j + 1
plt.show()

Let compute the decision boundaries, recall from the lecture that at the decision boundary:
$$
  z = 0 = w_1 x_1 + w_2 x_2 + w_0
$$
or 
$$
  x_2 = -\frac{1}{w_2}(w_1 x_1 + w_0 )
$$
lets compute this both for the true parameters and the estimated ones from the perceptron to compare.

In [ ]:
# compute decision boundary direction
x1_start = -2.5
x1_end = 2.5
x2_start = -(w0 + w1*x1_start)/w2
x2_end = -(w0 + w1*x1_end)/w2
x2_start_est = -(w0_hat + w1_hat*x1_start)/w2_hat
x2_end_est = -(w0_hat + w1_hat*x1_end)/w2_hat

Now, lets add these decision boundaries to the plot:

In [ ]:
# set up the plots
fig, (ax1, ax2) = plt.subplots(1, 2)
j = 1
for g in np.unique(y):
    ix = np.where(y == g)
    ax1.scatter(x1[ix], x2[ix], c = cdict[j], label = "y = " + str(g), s = 16, alpha=0.3)
    j = j + 1

j = 1
for g in np.unique(y_hat):    
    ix = np.where(y_hat == g)
    ax2.scatter(x1[ix], x2[ix], c = cdict[j], label = "$\hat{y} = $" + str(g), s = 16, alpha=0.3)
    j = j + 1

# plot the decision boundaries
ax1.plot([x1_start, x1_end], [x2_start, x2_end], 'k-')
ax2.plot([x1_start, x1_end], [x2_start_est, x2_end_est], 'k--')
ax1.legend()
ax2.legend()
ax1.set_xlim(-3, 3) 
ax2.set_xlim(-3, 3) 
ax1.set_ylim(-4, 4) 
ax2.set_ylim(-4, 4) 
ax1.set_xlabel("$x_1$")
ax1.set_ylabel("$x_2$")
ax2.set_xlabel("$x_1$")
ax1.set_title('Ground Truth')
ax2.set_title('Perceptron')
plt.show()

The two plots look near-identical so lets highlight the classification errors to make it easier:


In [ ]:
classification_error = (y!=y_hat) # compute which training samples are misclassified
fig2, ax = plt.subplots()
j = 1
for g in np.unique(classification_error):
    ix = np.where(classification_error == g)
    ax.scatter(x1[ix], x2[ix], c = cdict[j], label = "Error: " + str(g), s = 16, alpha=0.3)
    j = j + 1
ax.legend()
ax.plot([x1_start, x1_end], [x2_start, x2_end], 'k-')
ax.plot([x1_start, x1_end], [x2_start_est, x2_end_est], 'k--')
ax.set_xlim(-3, 3) 
ax.set_ylim(-4, 4) 
plt.xlabel("$x_1$")
plt.ylabel("$x_2$")
plt.show()

If the data can be entirely linearly separated, there will be no classification errors.


## Exercises

Modify the data simulation procedure to:

1. Increase/decrease the separation between the two classes, what happens to the performance of the classifier?
2. Vary the number of training samples, how does this alter the behaviour of the algorithm?
3. Can you plot a graph of degree of separation/number of training samples versus classifier performance?

How happens if you use a data generating procedure which is not the one assumed by the perceptron algorithm?

## Extended Exercises [Optional]

The following exercises are not compulsory but would test your understanding and should be done in your own time (they will not be covered in the labs): 

1.   Re-design the code to allow for any number of inputs (you will need to use matrix/vector operations provided by the numpy library).
2.   Read about the "Perceptron Convergence Theorem" (https://en.wikipedia.org/wiki/Perceptron#Convergence), show empirically that the performance of your code is consistent with the theorem.
3.   Apply your code to data from the UCI Data Repository (https://archive.ics.uci.edu/ml/datasets.php).